# [Appliances Energy Prediction](https://www.kaggle.com/loveall/appliances-energy-prediction)
## Data driven prediction of energy use of appliances

### Experimental data used to create regression models of appliances energy use in a low energy building. <br/>
Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters).

date time year-month-day hour:minute:second <br/>
Appliances, energy use in Wh<br/>
lights, energy use of light fixtures in the house in Wh<br/>
T1, Temperature in kitchen area, in Celsius<br/>
RH1, Humidity in kitchen area, in % <br/>
T2, Temperature in living room area, in Celsius <br/>
RH2, Humidity in living room area, in %<br/>
T3, Temperature in laundry room area<br/>
RH3, Humidity in laundry room area, in % <br/>
T4, Temperature in office room, in Celsius <br/>
RH4, Humidity in office room, in %<br/>
T5, Temperature in bathroom, in Celsius<br/>
RH5, Humidity in bathroom, in % <br/>
T6, Temperature outside the building (north side), in Celsius <br/>
RH6, Humidity outside the building (north side), in %<br/>
T7, Temperature in ironing room , in Celsius<br/>
RH7, Humidity in ironing room, in % <br/>
T8, Temperature in teenager room 2, in Celsius <br/>
RH8, Humidity in teenager room 2, in %<br/>
T9, Temperature in parents room, in Celsius<br/>
RH9, Humidity in parents room, in % <br/>
To, Temperature outside (from Chievres weather station), in Celsius <br/>
Pressure (from Chievres weather station), in mm Hg <br/>
RHout, Humidity outside (from Chievres weather station), in %<br/>
Wind speed (from Chievres weather station), in m/s<br/>
Visibility (from Chievres weather station), in km<br/>
Tdewpoint (from Chievres weather station), Â°C<br/>
rv1, Random variable 1, nondimensional<br/>
rv2, Random variable 2, nondimensional

## Importing Libraries


In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
# Models Applied
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

## Import Dataset

In [2]:
dataset_train=pd.read_csv('./train.csv')
dataset_train_copy=dataset_train.copy()

In [3]:
dataset_train_copy.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
dataset_train_copy.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [5]:
dataset_train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

## Data Preprocessing

In [6]:
# Check for NaN data
display(dataset_train_copy.isnull().sum())

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [7]:
corr=dataset_train_copy.corr()["Appliances"].abs().sort_values(ascending=False)
corr

Appliances     1.000000
lights         0.197278
RH_out         0.152282
T2             0.120073
T6             0.117638
T_out          0.099155
RH_8           0.094039
Windspeed      0.087122
RH_1           0.086031
T3             0.085060
RH_6           0.083178
RH_2           0.060465
RH_7           0.055642
T1             0.055447
RH_9           0.051462
T4             0.040281
T8             0.039572
RH_3           0.036292
Press_mm_hg    0.034885
T7             0.025801
T5             0.019760
RH_4           0.016965
Tdewpoint      0.015353
rv1            0.011145
rv2            0.011145
T9             0.010010
RH_5           0.006955
Visibility     0.000230
Name: Appliances, dtype: float64

### Training and Evaluating on the Training Set

In [8]:
#split dataset in features and target variable
feature_cols = ['RH_out','T2',
'T6','T_out','RH_8','Windspeed','RH_1',
'T3','RH_6','RH_2'
]
X = dataset_train_copy[feature_cols] # Features
y = dataset_train_copy.Appliances # Target variable

In [9]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) # 80% training and 20% test

### Feature scaling

In [10]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
def mean_relative_error(y_test,y_pred):
    error = 0
    n=len(y_test)
    for i in range(n):
        error += (abs(y_pred[i]-y_test[i])/y_test[i])
    return error/n

## Select and Train a Model

In [12]:
# Linear Regression
# create loocv procedure
cv = LeaveOneOut()
reg = LinearRegression().fit(X_train, y_train)
reg_pred = reg.predict(X_test)
print("Training set score: {:.2f}".format(reg.score(X_train, y_train)))
print("Test set score: {:.2f}".format(reg.score(X_test, y_test)))
print("Test set R^2: {:.2f}".format(reg.score(X_test, y_test)))
# evaluate model
scores = cross_val_score(
    reg,
    X_train,
    y_train,
    scoring="neg_mean_absolute_error",
    cv=cv
)
# force positive
scores = absolute(scores)
# report performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))
print("RMSE: {:.2f}".format(mean_squared_error(y_test, reg_pred, squared=False)))

Training set score: 0.12
Test set score: 0.11
Test set R^2: 0.11
MAE: 55.362 (79.324)
RMSE: 94.15


In [13]:
# KNN Regressor
neigh = KNeighborsRegressor(n_neighbors=5)
neigh = neigh.fit(X_train, y_train)
neigh_pred=neigh.predict(X_test)
print("Training set score: {:.2f}".format(neigh.score(X_train, y_train)))
print("Test set score: {:.2f}".format(neigh.score(X_test, y_test)))
print("Test set R^2: {:.2f}".format(neigh.score(X_test, y_test)))
#MAE-Lower values are better.
print("MAE: {:.2f}".format(mean_absolute_error(y_test, neigh_pred)))
print("RMSE: {:.2f}".format(mean_squared_error(y_test, neigh_pred, squared=False)))

Training set score: 0.68
Test set score: 0.50
Test set R^2: 0.50
MAE: 32.60
RMSE: 71.02


In [14]:
# SVR
svr = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1).fit(X_train,y_train)
svr_pred=svr.predict(X_test)
print("Training set score: {:.2f}".format(svr.score(X_train, y_train)))
print("Test set score: {:.2f}".format(svr.score(X_test, y_test)))
print("Test set R^2: {:.2f}".format(svr.score(X_test, y_test)))
print("MAE: {:.2f}".format(mean_absolute_error(y_test, svr_pred)))
print("RMSE: {:.2f}".format(mean_squared_error(y_test, svr_pred, squared=False)))

Training set score: 0.10
Test set score: 0.10
Test set R^2: 0.10
MAE: 42.07
RMSE: 94.99


In [15]:
# Random Forest Regressor
rfg = RandomForestRegressor(random_state=1).fit(X_train,y_train)
rfg_pred=rfg.predict(X_test)
print("Training set score: {:.2f}".format(rfg.score(X_train, y_train)))
print("Test set score: {:.2f}".format(rfg.score(X_test, y_test)))
print("Test set R^2: {:.2f}".format(rfg.score(X_test, y_test)))
print("MAE: {:.2f}".format(mean_absolute_error(y_test, rfg_pred)))
print("RMSE: {:.2f}".format(mean_squared_error(y_test, rfg_pred, squared=False)))

Training set score: 0.94
Test set score: 0.55
Test set R^2: 0.55
MAE: 31.94
RMSE: 66.89
